# Trabajo Práctico 1: Análisis Exploratorio de Datos
## Fecha y hora de entrega máxima:
11/04/2022 18:00

## Acerca del Dataset "Datos de clientes del banco"
Los datos están relacionados con campañas de marketing directo (llamadas telefónicas) de una institución bancaria portuguesa. El objetivo de la clasificación es predecir si el cliente suscribirá un depósito a plazo.

## 1. Listado de variables y selección
- Por cada variable explicar en una oración el contenido de dicha variable y definir si será utilizada como variable de entrada, de salida, o no será utilizada.
- Para la variable de salida especificar los valores posibles que puede tener.
- Por cada variable que no se vaya a utilizar, explicar brevemente el motivo.

<br />

|Variable|Descripción|Entrada|Salida|Sin utilización|
|--|---|---|--|--|
|age|Edad del cliente|X  |||
|job|Tipo de trabajo del cliente|X  |||
|marital|Estado civil del cliente|X  |||
|education|Tipo de educación del cliente|X  |||
|default|Indica si el cliente tiene crédito en mora|X  |||
|balance|Balance actual del cliente|X  |||
|housing|Indica si el cliente tiene un prestamo de vivienda|X  |||
|loan|Indica si el cliente tiene un prestamo personal|X  |||
|contact|Tipo de comunicación con el cliente|X  |||
|day|Día de la semana en que hubo contacto con el cliente por última vez |X  |||
|month|Mes del año en que hubo contacto con el cliente por últtima vez|X  |||
|duration|Duración en segundos del último contacto con el cliente|X  |||
|campaign|Número de contactos realizados al cliente durante la campaña|X  |||
|pdays|Número de días transcurridos desde la última vez que se contacto al cliente desde una campaña anterior|X  |||
|previous|Número de contactos realizados al cliente antes de la campaña actual|X  |||
|poutcome|Resultado de la campaña de marketing anterior|X  |||
|term_deposit|Indica si el cliente ha suscrito un depósito a término||X||

<br />

En cuanto a la variable de salida "term_deposit", los valores posibles que puede tener son un string binario "yes" o "no". No posee otro valor que éstos dos, con lo cual los datos de la variable de salida son utilizables sin necesidad de limpiarlos.

<br />

En cuanto a las variables que no se utilizarán, no elegimos ninguna, ya que consideramos que TODAS las variables de entrada posiblemente tengan una correlación con la variable de salida "term_deposit".

In [ ]:
# Importamos las dependencias necesarias.
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import keras
import tensorflow as tf
import h5py
import PIL
import sklearn_pandas
import plotly.express as px

In [ ]:
# If we import the dataset from the csv file we see we have values with the value "unknown"
ds = pd.read_csv("BankCustomerData.csv")

# Return the first five rows of the DataFrame
ds.head()

In [ ]:
# To replace these values with NaN, we must provide a list with all missing value formats
missing_value_formats = ["unknown", "n.a.","?","NA","n/a", "na", "--"]
ds = pd.read_csv("BankCustomerData.csv", na_values = missing_value_formats)

# Display the firsts and lasts lines of the file
ds

In [ ]:
# Get the length of the file
len(ds)

In [ ]:
# See dataset dimensions
ds.shape

In [ ]:
# Return the columns structure of the DataFrame
ds.dtypes

In [ ]:
# Return the NULL quantity values in each column
ds.isnull().sum()

## 2. Análisis detallado de un conjunto de variables
### 2.1 Para la variable de salida, explicar y graficar su balanceo y qué consecuencias va a tener eso a la hora de entrenar y medir el rendimiento de distintos modelos.

In [ ]:
# See percentage of yes and no in term_deposit as Bar Plot
ax = ds.term_deposit.value_counts().plot.bar()

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1, p.get_height() * 1))

In [ ]:
# See percentage of yes and no in term_deposit as Pie Plot
ds.term_deposit.value_counts().plot.pie()

In [ ]:
# Does not work with .value_counts(), so instead use len()
yes_counts = len(ds[ds.term_deposit == "yes"].term_deposit)
no_counts = len(ds[ds.term_deposit == "no"].term_deposit)
total_counts = len(ds)

print("Counts")
print("Yes counts: ", yes_counts)
print("No counts: ", no_counts)
print("Total counts: ", (yes_counts + no_counts))
print("")
print("Percentages")
print("Yes percentage: ", (yes_counts / total_counts) * 100)
print("No percentage: ", (no_counts / total_counts) * 100)

Como se puede observar en el gráfico de barras y en el de torta, existe un claro desequilibrio en los valores de la variable de salida "term_deposit", ya que el 90.5% de los clientes decidió no suscribirse a un depósito a plazo, mientras que un 9.5% sí lo hizo.

Debido a la gran cantidad de valores 'no', se podría llegar a afectar el entrenamiento y la medición del rendimiento del modelo, ya que podría aprender a predecir que en la mayoría de los casos, los clientes no se suscribirán a un depósito a plazo para evitar fallas.

### 2.2 Para 5 variables de entrada (elegidas o no, pero incluyendo al menos 3 elegidas) graficar y explicar cómo se comportan y cómo afectan a la variable de salida.

Para las variables de entrada, elegimos las siguientes 5 columnas:
- age
- balance
- loan
- housing
- contact

#### Variable age:

In [ ]:
px.histogram(ds, x='age', color='term_deposit', barmode='group')

In [ ]:
ds.groupby("term_deposit").aggregate({"age": "mean"})

In [ ]:
ds.groupby("term_deposit").aggregate({"age": "max"})

In [ ]:
ds.groupby("term_deposit").aggregate({"age": "min"})

En cuanto a la edad del cliente en relación a la variable term_deposit, podemos observar considerando el histograma, que esta variable de entrada guarda relación con la variable de salida term_deposit debido a que  se observa una tendencia alcista hasta la edad de 32 años, para luego ir decreciendo a lo largo del gráfico. 

Esto significa que el lograr que un cliente decida suscribirse o no a un depósito a plazo dependerá en gran medida de la edad de la persona.


Ésto podría suceder a raíz de que las personas que poseen cerca de 30 años poseen una capacidad de ahorro mayor, con lo cual la gran mayoría de las campañas para que un cliente decida suscribirse o no a un depósito a plazo están dirigidas hacia personas con una franza de edad de entre 30 y 50 años aproximadamente, con un promedio de edad de 40 años tanto para los "yes" como para los "no"

4 - Duration

In [ ]:
duration_no_pago_a_termino = ds[ds.term_deposit == "no"].duration.sum() / len(ds[ds.term_deposit == "no"].term_deposit)

duration_pago_a_termino = ds[ds.term_deposit == "yes"].duration.sum() / len(ds[ds.term_deposit == "yes"].term_deposit)

print('Duracion promedio de no pago a termino: ', duration_no_pago_a_termino)
print('Duracion promedio de pago a termino: ',  duration_pago_a_termino)

Los valores dicen que cuando pagan a termino en promedio el contacto dura mas del doble de cuando no pagan.

5 - Job

In [ ]:
px.histogram(ds, x='job', color='term_deposit', barmode='group')

In [ ]:
f,ax=plt.subplots(1,3,figsize=(16,6))

ax[0].set_title('Numero de veces que aparece cada valor')
valores_distintos = ds.job.value_counts()
valores_distintos.plot.bar(ax=ax[0])

ax[1].set_title('Porcentaje de veces que no paga a termino')
no_pago_a_termino = ds[ds.term_deposit == 'no'].job.value_counts() / ds.job.value_counts()
no_pago_a_termino.plot.bar(ax=ax[1])

ax[2].set_title('Porcentaje de veces que paga a termino')
pago_a_termino = ds[ds.term_deposit == 'yes'].job.value_counts() / ds.job.value_counts()
pago_a_termino.plot.bar(ax=ax[2])

De la variable Job podemos observar segun el histograma que al igual del resto de las varibles presenta un gran desbalanceo. Mirando los datos vemos que hay valores que se presentan mayor cantidad de veces en la muestra, lo cual puede hacer sobre entrenar al modelo o restarle importancia algun valor que no aparece tanto.

En los porcentajes, el valo 'student' es el que mas veces paga a termino seguido por 'retired' y 'unemployed'. Las primera dos tienen sentidos, ya que tanto un estudiante como una persona retirada tienen, en teoria, tiempo libre ya que no tienen responsabilidades laborales. En el caso del desempleado, tambien tiene sentido viendolo por ese lado, pero pierde un poco por el hecho de que tal vez no tenga dinero para pagar, aunque con esta variable sola no podemos verificar eso.

Por el lado de los que no pagan a termino, aparecen 'blue-collar', 'entrepreneur', 'housemaid', 'services'. Trabajos de diferentes indole que por algun motivo no pagan a tiempo.

### 2.3 Para cada una de las variables de entrada elegidas, explicar si se debería realizar o no alguna transformación para poder utilizarla como entrada de un modelo y por qué.

### 2.4 Analizar si las variables de entrada seleccionadas presentan valores nulos y/o extremos. En caso de encontrar dichas condiciones, indicar qué tratamiento se podría darle a las mismas y por qué.

### 2.5 Verificar si existen variables altamente correlacionadas con la variable "target". En dicho caso, explicar por qué considera que esto pasa.